# Emily and Michael's Research Notebook

## Add list of citations
https://ui.adsabs.harvard.edu/abs/2010ApJ...716....1B/abstract

https://iopscience.iop.org/article/10.1086/589678/pdf

https://ui.adsabs.harvard.edu/abs/2008ApJ...673..864J/abstract

## Import necessary packages


In [43]:
import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt

#This line is key if you want your plots to show up in the Jupyter notebook!
%matplotlib inline

## Import the data file

In [44]:
data = ascii.read('./data.dat')

In [45]:
data

ra,dec,run,Ar,u,g,r,i,z,uErr,gErr,rErr,iErr,zErr,pmL,pmB,pmErr
float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
181.6122,27.5726,5087,0.05,16.4,15.24,14.8,14.73,14.69,0.02,0.02,0.02,0.02,0.01,5.16,-1.45,2.72
181.61303,27.6077,5087,0.05,21.84,19.55,18.4,18.04,17.79,0.18,0.02,0.02,0.02,0.02,5.48,4.73,3.16
181.61416,27.35956,5112,0.05,19.76,17.56,16.61,16.23,16.07,0.03,0.01,0.02,0.03,0.03,3.14,7.92,2.79
181.6164,27.82464,5112,0.06,21.73,20.63,20.19,19.99,20.04,0.16,0.03,0.03,0.03,0.13,19.13,-1.08,5.08
181.62112,28.01623,5087,0.05,21.37,20.38,20.11,20.05,19.87,0.11,0.04,0.03,0.03,0.09,-3.65,-8.0,4.91
181.62252,27.63987,5087,0.05,17.34,15.58,14.82,14.64,14.51,0.02,0.02,0.02,0.02,0.01,9.74,-10.64,2.72
181.62469,27.32682,5112,0.05,21.63,20.52,20.21,20.1,19.91,0.13,0.02,0.03,0.04,1.07,6.73,-25.39,5.13
181.62582,27.42162,5112,0.05,23.16,21.09,19.67,19.04,18.66,0.46,0.03,0.02,0.03,0.04,35.17,7.8,4.18
181.63178,26.93836,5112,0.05,18.57,15.96,14.77,14.25,13.96,0.02,0.02,0.02,0.02,0.02,15.96,-8.01,2.72


In [50]:
g_minus_r = np.array(data['g'] - data['r'])
u_minus_g = np.array(data['u'] - data['g'])

def calc_metallicity(x, y):
    '''
    Calculate the metallicity of an array of stars
    
    x - The u-g value of the star as an array
    y - The g-r value of the stars as an array
    return - An array of metallicities
    '''
    
    #Calculate the masks
    result = np.zeros(len(y));
    mask = y < 0.6;
    
    #Update the values that are less than 0.6
    A = [-13.13, 14.09, 28.04, -5.51, -5.90, -58.68, 9.14, -20.61, 0.0, 58.20]
    result[mask] = A[0] + A[1]*x[mask] + A[2]*y[mask] + A[3]*x[mask]*y[mask] + A[4]*x[mask]**2 + A[5]*y[mask]**2 + A[6]*(x[mask]**2)*y[mask] + A[7]*x[mask]*y[mask]**2 + A[8]*x[mask]**3 + A[9]*y[mask]**3
        
    #Set all values greater than 0.6 to -0.6
    result[np.invert(mask)] = -0.6
    
    #Return the final value
    return result

metallicity = calc_metallicity(u_minus_g, g_minus_r)
print("g-r:", g_minus_r[:10])
print("metallicity:", metallicity[:10])

g-r: [0.44 1.15 0.95 0.44 0.27 0.76 0.31 1.42 1.19 0.98]
metallicity: [-0.8191296  -0.6        -0.6        -1.0249488  -1.06648793 -0.6
 -0.57584997 -0.6        -0.6        -0.6       ]


In [51]:
#Define functions to calculate Mr
def calc_delta_Mr(metallicity):
    return 4.50 - 1.11*metallicity - 0.18*metallicity**2
    
def calc_Mr0(g_minus_i):
    return -5.06 + 14.32*g_minus_i - 12.97*g_minus_i**2 + 6.127*g_minus_i**3 - 1.297*g_minus_i**4 + 0.0967*g_minus_i**5
    
def calc_Mr(g_minus_i, metallicity):
    '''
    Calculate the absolute magnitude of a list of stars
    
    g_minus_i - The g - r values of the stars as an array
    metallicity - The metallicity of the stars as an array
    return - The absolute magnitude in the r band as an array
    '''
    
    return Mr0(g_minus_i) - delta_Mr(metallicity)

g_minus_i = np.array(data['g'] - data['i'])
Mr = calc_Mr(g_minus_i, metallicity)
print(Mr[:10])

[-5.69041125 -2.99973111 -3.29952371 -5.25737268 -7.02072019 -4.01336671
 -5.99817063 -1.93204966 -2.63924016 -3.18439139]


In [48]:
#Find the distance to the star in Parsec
def calc_distance(r, Mr):
    '''
    Calculate the distance to stars in parsecs
    
    r - The r-band visual magnitude of the stars as an array
    Mr - The r-band absolute magnitude of the stars as an array
    return - The distances to the stars in parsecs as an array
    '''
    return 10**((r - Mr)/5 + 1)

distances = calc_distance(data['r'], Mr)
print(distances[:10])

        r         
------------------
125337.85254061809
190522.47823420833
  95919.0218263608
1228781.1375409649
 2667743.299112532
 58435.03961935261
1744351.9937198407
209126.91527997964
  30328.2975453213
157354.17792704195
